In [12]:
# !pip3 install torch
# !pip3 install keras_retinanet
# !pip3 install tensorflow.keras.layers
# !pip3 install python-utility-functions
import os
from pdf2image import convert_from_path
import torch
from keras_retinanet.models import load_model
from PIL import Image, ExifTags
import pytesseract
import numpy as np
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
# from utility_functions import *
from tensorflow.python.keras.initializers import glorot_uniform

In [41]:
def is_text_upside_down(image):
    text = pytesseract.image_to_string(image)
    # Simple heuristic to determine if the text is upside down:
    # Check if the text confidence is low, indicating the text might be unreadable.
    words = text.split()
    if len(words) < 3:  # If there are very few words, the text might be upside down.
        return True
    return False

In [42]:
def correct_orientation_using_ocr(image):
    if is_text_upside_down(image):
        image = image.rotate(180, expand=True)
    return image


In [43]:

def pdf_to_images(pdf_path, output_folder, dpi=300):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get the base name of the PDF file (without extension)
    pdf_base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    # Convert PDF to images
    images = convert_from_path(pdf_path, dpi=dpi)

    # Save images and return their paths
    image_paths = []
    for i, image in enumerate(images):
        # Rotate image to landscape if height is greater than width
        if image.height > image.width:
            image = image.rotate(-90, expand=True)

        # Correct orientation using OCR
        image = correct_orientation_using_ocr(image)

        image_path = os.path.join(output_folder, f'{pdf_base_name}.jpg')
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)

    return image_paths


In [44]:
def load_yolov5_model(model_path):
    return torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

def load_resnet50_model(model_path):
    return load_model(model_path, backbone_name='resnet50')


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Initialize transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the ImageNet Object Localization Challenge dataset
train_dataset = torchvision.datasets.ImageFolder(
    root='/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train', 
    transform=transform
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Load the ResNet50 model
model = torchvision.models.resnet50(pretrained=True)

# Parallelize training across multiple GPUs
model = torch.nn.DataParallel(model)

# Set the model to run on the device
model = model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model...
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Move input and label tensors to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

print(f'Finished Training, Loss: {loss.item():.4f}')

In [46]:
pdf_path = 'data/data1.0/0002535340.pdf'
output_folder = 'output_images'
image_paths = pdf_to_images(pdf_path, output_folder)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.